# **London bike sharing trends**

## **Business Understanding:**
This notebook aims to analyze and manipulate bike share trends using Transport for London's (TfL) free transport data service. The goal is to gain insights into factors influencing bike share demand and behavior, leveraging data from cycling datasets, weather information from freemeteo.com, and UK bank holidays data.

## **Data Understanding**:
The primary data sources include:

TfL's cycling dataset from https://cycling.data.tfl.gov.uk/, powered by TfL Open Data.
Weather data from freemeteo.com.
UK bank holidays data from https://www.gov.uk/bank-holidays.
The cycling dataset is organized with the following metadata:

- "timestamp": Timestamp field for grouping the data.
- "cnt": Count of new bike shares.
- "t1": Real temperature in Celsius.
- "t2": Temperature in Celsius (feels like).
- "hum": Humidity in percentage.
- "wind_speed": Wind speed in km/h.
- "weather_code": Category of the weather.
- "is_holiday": Boolean field (1 for holiday, 0 for non-holiday).
- "is_weekend": Boolean field (1 if the day is a weekend).
- "season": Category field for meteorological seasons (0-spring; 1-summer; 2-fall; 3-winter).
# **Problem Statement**:
The objective is to conduct a comprehensive analysis of bike share trends and identify patterns in bike share demand. The focus is on understanding how various factors such as weather conditions, holidays, weekends, and seasons impact bike share usage. The goal is not prediction but to provide descriptive insights for optimizing bike share services and resources.

In [18]:
# importing requried libraries 
import pandas as pd

In [19]:
# importing the data using pandas
df = pd.read_csv("./london_merged.csv")
# View of the first 5 rows
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [20]:
# View of the last 5 rows
df.tail()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0
17413,2017-01-03 23:00:00,139,5.0,1.0,76.0,22.0,2.0,0.0,0.0,3.0


In [21]:
# checking colum names
df.columns

Index(['timestamp', 'cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code',
       'is_holiday', 'is_weekend', 'season'],
      dtype='object')

In [22]:
# checking data types of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [23]:
# checking for null values in our dataset
df.isna().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

In [24]:
# checking for duplicate values
df.duplicated().sum()

0

In [25]:
# renaming columns to predered column names 
new_names_dict = {
    'timestamp':"time", 
    'cnt':'count', 
    't1':"real_temp", 
    't2':"temperature_feels", 
    'hum':'humidity_percentage', 
    'wind_speed':'wind_speed', 
    'weather_code':'weather_code',
    'is_holiday':'is_holiday', 
    'is_weekend':'is_weekend', 
    'season':'season'
}

df.rename(new_names_dict, axis=1, inplace=True)

In [26]:
# changing humidity from percentage to actual humidity
df['humidity_percentage'] = df['humidity_percentage'] / 100

In [27]:
# checking for unique values in season
df.season.value_counts()

0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: season, dtype: int64

In [28]:
# checking for unique values in is_holiday
df['is_holiday'].value_counts()

0.0    17030
1.0      384
Name: is_holiday, dtype: int64

In [29]:
# checking for unique values in is_weekend
df['is_weekend'].value_counts()

0.0    12444
1.0     4970
Name: is_weekend, dtype: int64

In [30]:
# checking for unique values in weather_code
df['weather_code'].value_counts()

1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: weather_code, dtype: int64

In [31]:
# maping actual values to integers in this 
season_names_dict = {
    '0.0' :'spring',
    '1.0': 'summer',
    '2.0' : 'autumn',
    '3.0' : 'winter'
}
weather_code_dict = {
    '1.0':'clear',
    '2.0':'scattered clouds', 
    '3.0':'broken clouds', 
    '7.0':'cloudy', 
    '4.0':'rain', 
    '26.0':'rain and thunderstrom', 
    '10.0':'snow'
}
# converting seasons datatype form integer to string
df.season = df.season.astype(str)
# maping values 
df['season'] = df['season'].map(season_names_dict)

# converting weather_code datatype form integer to string
df.weather_code = df.weather_code.astype(str)
# maping values
df['weather_code'] = df['weather_code'].map(weather_code_dict)

In [32]:
# Verifying whether the alterations are significant
df.head()

,time,count,real_temp,temperature_feels,humidity_percentage,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,clear,0.0,1.0,winter


In [34]:
# Converting the document into Excel format for visualization in Tableau.
df.to_excel('bikes.xlsx', sheet_name='Data')